In [78]:
import pandas as pd
import numpy as np

## Create a reproducible sample of a 1000 questions in each category

In [95]:
hops = [1,2,3]
for hop in hops:
    mqa = pd.read_csv(f"/datasets/MetaQA/{hop}hop/qa_test.txt", sep="\t", header=None)
    np.random.seed(42)
    mqa = mqa.sample(n=1000)
    mqa.to_csv(f"/datasets/MetaQA/{hop}hop/test_1000.txt", header=False)